In [1]:
############################################################################################
# Init
############################################################################################

import __init__

# enable autoreload (see https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
from util.loadUtil import loadDbuxFile, collectionDf
from data.DataProvider import DataProvider

from IPython.display import display, HTML

import re
from collections import Counter

%matplotlib inline 
sns.set(color_codes=True)

In [2]:
# load all data
data = DataProvider('item_data.json')

In [3]:
# get stat of function names and occurrence
def getStaticContextStat(dp):
    weight = dp.collections.contexts.groupby('staticContextId').size()
    weight = pd.DataFrame(weight).rename(columns = { 0:'occurrence' })

    function_names = dp.collections.staticContexts[['staticId', 'type', 'name']]
    function_names = function_names.rename(columns = { 'staticId':'staticContextId' })

    functions = pd.merge(function_names, weight, how='left', on='staticContextId').fillna(0)
    functions['name'] = functions['name'].astype(str)
    functions['occurrence'] = functions['occurrence'].astype('int32')
    return functions

functions = getStaticContextStat(data)

In [4]:
def removeExtName(df):
    for index, row in df.iterrows():
        if row['type'] == 1:
            df['name'][index] = re.sub("\..*$", "", row['name'])
    return df

functions = removeExtName(functions)

<>:4: DeprecationWarning: invalid escape sequence \.
C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
# generate keywords from name by spliting camelCase and non-alphabet char
def name2keyword(s):
    s = re.sub("([a-z])([A-Z])", "\g<1> \g<2>", s)
    s = re.sub("([^a-zA-Z])", " ", s)
    s = s.lower()
    s = s.split(' ')
    s = filter(None, s)
    return Counter(s)

functions = functions.assign(
    keywords = lambda df: df['name'].map(name2keyword)
)
display(functions)

<>:3: DeprecationWarning: invalid escape sequence \g


,staticContextId,type,name,occurrence,keywords
0,1,1,item,1,{'item': 1}
1,2,1,store,1,{'store': 1}
2,3,2,constructor,1,{'constructor': 1}
3,4,2,getLocalStorage,6,"{'get': 1, 'local': 1, 'storage': 1}"
4,5,2,setLocalStorage,2,"{'set': 1, 'local': 1, 'storage': 1}"
...,...,...,...,...,...
69,70,2,0,0,{}
70,71,2,_filter,8,{'filter': 1}
71,72,2,0,3,{}
72,73,1,app,3,{'app': 1}


In [6]:
# sum up occurrences of keywords
keywords = Counter()
for index, row in functions.iterrows():
    new_words = row['keywords']
    times = row['occurrence']
    for k in new_words.keys():
        keywords[k] += new_words[k] * times

keywordsDF = pd.DataFrame(keywords.most_common())
keywordsDF = keywordsDF.rename(columns={0:'keyword', 1:'occurrence'})
display(keywordsDF)

,keyword,occurrence
0,item,18
1,set,18
2,on,18
3,delegate,18
4,find,12
5,qs,12
6,filter,9
7,bind,9
8,local,8
9,storage,8
